In [4]:
# -*- coding: utf8 -*-

In [5]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
conf = SparkConf().setAppName("hospitalDiseaseWorr2vec")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [6]:
import numpy as np
from gensim.models import Word2Vec
from medicalDataLoad import *

In [7]:
#RF model train and test
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from time import time

### load一级分类index和名称词典

In [16]:
def loadCodeType(fileName):
    fileList = list(open(fileName,"r").readlines())
    #print(fileList)
    firstCode2IndexDict = dict()
    firstIndex2TypeDict = dict()
    for i in range(len(fileList)):
        itemArr = fileList[i].strip().split(" ")
        firstCode2IndexDict[itemArr[0]] = itemArr[1]
        firstIndex2TypeDict[itemArr[1]] = itemArr[2]
    return firstCode2IndexDict, firstIndex2TypeDict
fileName = "datasets/firstCode2Index2Type.txt"
firstCode2IndexDict, firstIndex2TypeDict = loadCodeType(fileName)
#print(firstCode2IndexDict)
print(firstIndex2TypeDict)

{'17': '症状、体症和临床与实验室异常所见，不可归类在他处者', '3': '内分泌、营养和代谢疾病', '4': '精神和行为障碍', '11': '皮肤和皮下组织病', '6': '眼和附器疾病', '18': '损伤、中毒和外因的某些其它后果', '15': '起源于围生期的某些情况', '10': '消化系统疾病', '16': '先天畸形、变性和染色体异常', '8': '循环系统疾病', '9': '呼吸系统疾病', '5': '神经系统疾病', '2': '血液及造血器官疾病和某些涉及免疫机制的疾患', '12': '肌肉和骨骼系统和结缔组织疾病', '19': '疾病和死亡的外因', '14': '妊娠分娩及产褥期', '0': '某些传染病和寄生虫病', '7': '耳和乳突疾病', '20': '影响健康状态和保健机构接触的因素', '1': '肿瘤', '13': '泌尿生殖系统疾病'}


### 输入keyword，输出词向量的均值

In [17]:
#4.word2vec
def keyWord2Vec(word, model,vector_size):
    disNoList = list()
    test_docs = word.split(" ")
    vecSum=np.zeros(vector_size)
    realLen =0
    for word in test_docs:
        try:
            #print(word)
            vecTmp = model[word]
            vecSum += vecTmp
            realLen +=1
        except KeyError:
            disNoList.append(word)
    #print("vec=",vecSum)
    return vecSum*1.0/realLen

### 输入向量list，输出预测结果

In [18]:
def RFPredict(rfModel, inListKey):
    inRDD = sc.parallelize(inListKey) 
    predict = rfModel.predict(inRDD)
    for item in predict.collect():
        print (int(item), firstIndex2TypeDict[str(int(item))])
    res = [firstIndex2TypeDict[str(int(item))] for item in predict.collect()]
    print(res)
    return  predict

### 输入disease+admin keyword，输出向量

In [27]:
def doPrepareKeywordDiseaseAndAdminAve(dis, admin, word2vec_model,vector_size):
    disVec = keyWord2Vec(dis,word2vec_model,vector_size)
    adminVec = keyWord2Vec(admin,word2vec_model,vector_size)
    vecRow = (disVec + adminVec)*1.0/2 
    return vecRow  

### 输入关键字list，输出向量list

In [30]:
def doPrepareKeyList2VecList(inList):
    vector_size = 100
    modelName = "model/word2VecModelsh.bin10_100_1e-05_15"
    word2vec_model = Word2Vec.load(modelName) 
    vecList = list()
    for i in range(len(inList)):
        item = inList[i]
        itemArr = item.split(" ")
        dis = itemArr[0]
        admin = " ".join(itemArr[1:])
        vec = doPrepareKeywordDiseaseAndAdminAve(dis, admin, word2vec_model,vector_size)
        vecList.append(vec)
    return vecList

In [35]:
def testRFKeyword():
    inList = list() #皮肤  
    inStr1 = "大疱性类天疱疮 水疱 散在 疱液 四肢 红斑 黄豆 大小 纤维 鳞屑性丘疹 经治疗 轻度瘙痒 密集分布 潍坊市皮肤病医院 融合 钟台村委会 电镜检查 点状出血现象 疗效不佳 双侧 "#神经系统疾病
    inList.append(inStr1)
    inStr2 = "气管恶性肿瘤 湿性啰音 肝脾 无胸痛 查体 大小便 心率 下肢 失眠 发作 饮食" #0
    inList.append(inStr2)
    inStr3 = "虹膜异色性白内障 前房深度 晶状体混浊 虹膜 眼底 角膜 瞳孔 充血 水肿 模糊 直径 透明网膜 右眼结膜 B超 玻璃体混浊 玻璃体后脱离 右眼视力下降 眼结膜 不清 在位"#眼和附器疾病
    inList.append(inStr3)
    inStr4 = "脑梗死 视物不清 双侧瞳孔 心肺听诊 肌力肌张力 肢体活动 无恶心呕吐 视野缺损 等大 等圆 无压痛 查体 左眼 神志 不灵 食欲 未及 头晕 "#循环系统疾病
    inList.append(inStr4)
    inStr5="膜炎 大便 患者 CT 周围脓肿形成 胆囊结石 伴胆囊炎 炎性病变 腹部疼痛 全腹 阑尾穿孔 排气排便 消化道穿孔 肾周围 腹腔粘连 自服药物 科会 阑尾粪石 左肾结石 阑尾炎 下腹部 腹膜"#消化
    inList.append(inStr5)


    savePath  = "datasets/w2v_rf_model2"
    rfModel = RandomForestModel.load(sc, savePath)
 
    inListVec = doPrepareKeyList2VecList(inList )
    res = RFPredict(rfModel, inListVec)
testRFKeyword()

11 皮肤和皮下组织病
1 肿瘤
6 眼和附器疾病
8 循环系统疾病
10 消化系统疾病
['皮肤和皮下组织病', '肿瘤', '眼和附器疾病', '循环系统疾病', '消化系统疾病']


### 入院情况输入原始文本，输出向量 

In [20]:

def doPrepareSourceDiseaseAndAdminAve(dis, admin, word2vec_model,vector_size):
    adminKey = getKeyWordFromSource(admin)
    disVec = keyWord2Vec(dis,word2vec_model,vector_size)
    adminVec = keyWord2Vec(adminKey,word2vec_model,vector_size)
    vecRow = (disVec + adminVec)*1.0/2 
    return vecRow

In [21]:
def testRFSource():
    vector_size = 100
    modelName = "model/word2VecModelsh.bin10_100_1e-05_15"
    word2vec_model = Word2Vec.load(modelName) 
    
    inListKey = list()
    inSource1Dis = "脑梗死"#循环
    inSource1Adm = "伸舌居中。颈软，无抵抗感，脑膜刺激征（-），心肺听诊无异常。右侧肢体肌力4+级，生理反射存在，病理征未引出。2015-02-23头颅MRI示脑梗死"
    inListKey.append(doPrepareSourceDiseaseAndAdminAve(inSource1Dis,inSource1Adm, word2vec_model, vector_size ))
    
    inSource2Dis = "胃恶性肿瘤"#肿瘤
    inSource2Adm = "患者1年半（2015年1月）前无明显原因及诱因出现上腹疼痛，后就诊于当地医院，行上腹部CT及胃镜检查，诊断“胃癌”，遂行胃癌手术治疗(具体手术范围不详)，术中发现肝脏转移结节，术后病理示“胃高分化腺癌”，术后未行放化疗，病情好转出院。患者于1月前无明显原因及诱因出现胸闷不适，无心悸、胸痛及肩背部放射痛，无发热、咳嗽、咳痰、咯血。为求进一步治疗就诊于我院，门诊以“胃癌术后”收入我科，患者自发病以来，神志清，精神好，饮食睡眠一般，大小便正常。"
    inListKey.append(doPrepareSourceDiseaseAndAdminAve(inSource2Dis,inSource2Adm,word2vec_model, vector_size ))
    
    savePath  = "datasets/w2v_rf_model2"
    rfModel = RandomForestModel.load(sc, savePath)
    res = RFPredict(rfModel, inListKey)

testRFSource()

8 循环系统疾病
1 肿瘤
['循环系统疾病', '肿瘤']
